In [1]:
from pathlib import Path
import pandas as pd
import sys
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import (RBF, ConstantKernel as C,
                                              Matern, WhiteKernel, DotProduct)
from sklearn.metrics import pairwise_distances, mean_squared_error,r2_score, mean_absolute_percentage_error
from sklearn.decomposition import PCA
from scipy.stats import pearsonr
from pathlib import Path
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

In [2]:
creep_df = pd.read_csv('Ti_alloys_dataset.csv')
creep_df

,Ti,Al,V,Fe,C,N,H,O,Sn,Nb,...,Cr,Solution treated temp(cel),ST time (h),Anneal temp (cel),Annealing Time (hour),Temperature of creep test (cel),Stress (Mpa),steady state strain rate (1/s),Strain to rupture (%) (Efc),creep_rupture_life
0,87.8750,6.75,4.50,0.40,0.100,0.050,0.1250,0.20,0.00,0.0,...,0.0,690,4.0,1050,0.5,700,319.0,2.090000e-03,15.80,0.01000
1,87.8100,6.61,4.23,1.18,0.026,0.011,0.0030,0.13,0.00,0.0,...,0.0,0,0.0,0,0.0,600,319.0,3.240000e-06,0.00,0.01167
2,87.8100,6.61,4.23,1.18,0.026,0.011,0.0030,0.13,0.00,0.0,...,0.0,0,0.0,0,0.0,600,250.0,4.400000e-05,0.00,0.03050
3,87.8100,6.61,4.23,1.18,0.026,0.011,0.0030,0.13,0.00,0.0,...,0.0,0,0.0,0,0.0,500,520.0,4.430000e-05,0.00,0.04620
4,87.8100,6.61,4.23,1.18,0.026,0.011,0.0030,0.13,0.00,0.0,...,0.0,0,0.0,0,0.0,700,319.0,2.130000e-03,0.00,0.05000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
172,85.9870,6.20,0.00,0.00,0.021,0.004,0.0080,0.03,1.95,0.0,...,0.0,900,1.0,580,8.0,520,310.0,1.200000e-04,0.00,1160.00000
173,89.0434,6.51,4.08,0.16,0.010,0.005,0.0016,0.19,0.00,0.0,...,0.0,0,0.0,940,4.0,455,379.0,9.160000e-07,0.00,1619.00000
174,89.0434,6.51,4.08,0.16,0.010,0.005,0.0016,0.19,0.00,0.0,...,0.0,0,0.0,1030,0.5,455,379.0,1.270000e-06,0.00,1744.00000
175,89.4100,6.00,4.00,0.25,0.080,0.050,0.0100,0.20,0.00,0.0,...,0.0,0,0.0,0,0.0,538,103.0,9.000000e-07,22.30,4681.00000


In [3]:
rm_state = 123
test_size = 0.6

X, X_test, y, y_test = train_test_split(np.array(creep_df.iloc[:, 0:12]), np.array(creep_df.iloc[:,12]), shuffle=True, test_size=test_size, random_state=rm_state)

In [4]:
print(X.shape)
print(X_test.shape)
print(y.shape)
print(y_test.shape)

(70, 12)
(107, 12)
(70,)
(107,)


In [5]:
idx = np.arange(len(y))

In [6]:
len(idx)

70

In [7]:
train_ratio = 0.1

X_train, _, y_train, _, idx_train, idx_pool = train_test_split(X, y, idx, train_size=train_ratio, shuffle=True, random_state=rm_state)

In [8]:
print(X_train.shape)
print(y_train.shape)

(7, 12)
(7,)


In [9]:
print(idx_train.shape)
print(idx_pool.shape)

(7,)
(63,)


In [10]:
n_iter = 6
pcc_variance = []
r2_variance = []
rmse_variance = []
mae_variance = []

pcc_random = []
r2_random = []
num_training_data=[]

kernel = C(1.0) * Matern(length_scale=1.0) + WhiteKernel(noise_level=1.0) + C(1.0) * DotProduct(sigma_0=1.0)
model = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=8, normalize_y=True)

X_train_var = X_train
X_train_ran = X_train
y_train_var = y_train
y_train_ran = y_train
idx_pool_var = idx_pool
idx_pool_ran = idx_pool
idx_train_var = idx_train
idx_train_ran = idx_train

for i in range(n_iter):
    print(f"Performing iteration : {i}")

    if i != 0:
        # find 10 data points with the highest variance
        q_points_var = np.argpartition(y_pred_unc_pool_var, -10)[-10:]
        # indices of those points in idx_pool
        idx_pool_train_var = idx_pool_var[q_points_var]

        idx_train_var = np.append(idx_train_var, idx_pool_train_var)
        idx_pool_var = np.delete(idx_pool_var, q_points_var)
        X_train_var = X[idx_train_var]
        y_train_var = y[idx_train_var]

    print(f"Number of training data with variance: {len(idx_train_var)}")
    print(f"Number of pooling data with variance: {len(idx_pool_var)}")

    num_training_data.append(len(idx_train_var))

    model.fit(X_train_var, y_train_var)

    y_pred_test_var, y_pred_unc_test_var = model.predict(X_test, return_std=True)
    y_pred_train_var, y_pred_unc_train_var = model.predict(X_train_var, return_std=True)
    y_pred_pool_var, y_pred_unc_pool_var = model.predict(X[idx_pool_var], return_std=True)
    

    print('PCC_test', pearsonr(y_test, y_pred_test_var)[0])
    print('R2_test', r2_score(y_test, y_pred_test_var))
    print('RMSE', np.sqrt(mean_squared_error(y_test, y_pred_test_var)))
    print('MAE', np.mean(abs(y_test - y_pred_test_var)))

    
    pcc_variance.append(pearsonr(y_test, y_pred_test_var)[0])
    r2_variance.append(r2_score(y_test, y_pred_test_var))
    rmse_variance.append(np.sqrt(mean_squared_error(y_test, y_pred_test_var)))
    mae_variance.append(np.mean(abs(y_test - y_pred_test_var)))

Performing iteration : 0
Number of training data with variance: 7
Number of pooling data with variance: 63
PCC_test 0.43295988569976296
R2_test -0.13647835626523075
RMSE 0.13158296819580695
MAE 0.08791972518829975
Performing iteration : 1
Number of training data with variance: 17
Number of pooling data with variance: 53


c:\Users\longz\anaconda3\envs\pytorchcpu\Lib\site-packages\sklearn\gaussian_process\_gpr.py:629: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  _check_optimize_result("lbfgs", opt_res)
c:\Users\longz\anaconda3\envs\pytorchcpu\Lib\site-packages\sklearn\gaussian_process\kernels.py:420: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__k2__noise_level is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
c:\Users\longz\anaconda3\envs\pytorchcpu\Lib\site-packages\sklearn\gaussian_process\kernels.py:420: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__k1__constant_value is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.

PCC_test 0.4068118567125323
R2_test -0.14975192784766533
RMSE 0.1323491533556823
MAE 0.07822339798447446
Performing iteration : 2
Number of training data with variance: 27
Number of pooling data with variance: 43
PCC_test 0.9751555128200649
R2_test 0.949478025146762
RMSE 0.02774337252323331
MAE 0.011848347845178027
Performing iteration : 3
Number of training data with variance: 37
Number of pooling data with variance: 33


c:\Users\longz\anaconda3\envs\pytorchcpu\Lib\site-packages\sklearn\gaussian_process\_gpr.py:629: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  _check_optimize_result("lbfgs", opt_res)
c:\Users\longz\anaconda3\envs\pytorchcpu\Lib\site-packages\sklearn\gaussian_process\kernels.py:420: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__k2__noise_level is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(
c:\Users\longz\anaconda3\envs\pytorchcpu\Lib\site-packages\sklearn\gaussian_process\kernels.py:420: ConvergenceWarning: The optimal value found for dimension 0 of parameter k2__k1__constant_value is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.

PCC_test 0.9774498148752805
R2_test 0.9551973498976587
RMSE 0.02612588109725223
MAE 0.007573623767897518
Performing iteration : 4
Number of training data with variance: 47
Number of pooling data with variance: 23


c:\Users\longz\anaconda3\envs\pytorchcpu\Lib\site-packages\sklearn\gaussian_process\kernels.py:420: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__k2__noise_level is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(


PCC_test 0.9773053360372027
R2_test 0.9549174703724415
RMSE 0.026207357473617323
MAE 0.007602088953176315
Performing iteration : 5
Number of training data with variance: 57
Number of pooling data with variance: 13
PCC_test 0.9775031117600308
R2_test 0.9553121726909527
RMSE 0.02609238116868175
MAE 0.007550596466376209


c:\Users\longz\anaconda3\envs\pytorchcpu\Lib\site-packages\sklearn\gaussian_process\kernels.py:420: ConvergenceWarning: The optimal value found for dimension 0 of parameter k1__k2__noise_level is close to the specified lower bound 1e-05. Decreasing the bound and calling fit again may find a better value.
  warnings.warn(


'\n    # random sampling\n    if i != 0:\n        # select 10 random data points\n        q_points_ran = np.random.choice(np.arange(len(idx_pool_ran)), size=10)\n        # indices of those points in idx_pool\n        idx_pool_train_ran = idx_pool_ran[q_points_ran]\n\n        idx_train_ran = np.append(idx_train_ran, idx_pool_train_ran)\n        idx_pool_ran = np.delete(idx_pool_ran, q_points_ran)\n        X_train_ran = X[idx_train_ran]\n        y_train_ran = y[idx_train_ran]\n\n    print(f"Number of training data with random: {len(idx_train_ran)}")\n    print(f"Number of pooling data with random: {len(idx_pool_ran)}")\n\n    model.fit(X_train_ran, y_train_ran)\n\n    y_pred_test_ran, y_pred_unc_test_ran = model.predict(X_test, return_std=True)\n    y_pred_train_ran, y_pred_unc_train_ran = model.predict(X_train_ran, return_std=True)\n    y_pred_pool_ran, y_pred_unc_pool_ran = model.predict(X[idx_pool_ran], return_std=True)\n\n    print(\'PCC_test\', pearsonr(y_test, y_pred_test_ran)[0])\

In [11]:
import pickle
with open('AL_GPR_Ti.pkl', 'wb') as f:
    pickle.dump({'train_numbs':num_training_data, 'pcc':pcc_variance,'r2':r2_variance, 'rsme': rmse_variance, 'mae': mae_variance}, f)
    f.close()

pkl_file = open('AL_GPR_Ti.pkl', 'rb')  
test_ALGPR = pickle.load(pkl_file)